In [1]:
%matplotlib inline
#import pycld2
import sys
sys.path

['',
 '/usr/lib/python35.zip',
 '/usr/lib/python3.5',
 '/usr/lib/python3.5/plat-x86_64-linux-gnu',
 '/usr/lib/python3.5/lib-dynload',
 '/home/ira/.local/lib/python3.5/site-packages',
 '/usr/local/lib/python3.5/dist-packages',
 '/usr/local/lib/python3.5/dist-packages/setuptools-28.8.0-py3.5.egg',
 '/usr/lib/python3/dist-packages',
 '/home/ira/.local/lib/python3.5/site-packages/IPython/extensions',
 '/home/ira/.ipython']

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup
from polyglot.detect import Detector

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize

In [3]:
DATA = "data/"

df = pd.read_csv(DATA+"Сушия.csv")

In [4]:
df.head()

,title,text,keyword_role,keyword_tone,source_type,source_url
0,Какой у вас в Харькове Сушия? У нас в Одессе к...,<p>Какой у вас в Харькове Сушия? У нас в Одесс...,object,negative,socialnetworksource,http://instagram.com/
1,"Меня качество еды в Сушия устраивает, но не ве...","<p>Меня качество еды в Сушия устраивает, но не...",object,negative,socialnetworksource,http://youtube.com/
2,"[club30459962|Сушия – сучасна японська кухня],...",<p>[club30459962|Сушия – сучасна японська кухн...,object,negative,socialnetworksource,http://vk.com/
3,​Ни при каких не заказывайте доставку,<p> Отзыв о Сушия: Ни при каких не заказывайте...,object,negative,socialnetworksource,http://www.otzyvua.net/
4,"Сушия, на час не на 2.","<p>Сушия, на час не на 2.</p>",object,negative,socialnetworksource,http://vk.com/


In [5]:
df['keyword_tone'].value_counts()

neutral     3228
negative    3228
positive    3228
Name: keyword_tone, dtype: int64

In [6]:
symbols_in_text = df['text'].str.len()

In [7]:
symbols_in_text.value_counts().head()

198.0    351
18.0     150
167.0    118
37.0      91
30.0      87
Name: text, dtype: int64

Найдовші тексти

In [8]:
df[symbols_in_text == 351]['text']

2534    <p>Сегодня в центре Киева, в ресторания ""Суши...
2557    <p>Сегодня в центре Киева, в ресторания ""Суши...
2724    <p>В центре Киева расстреляли посетителя ресто...
2807    <p>Сегодня в центре Киева, в ресторания ""Суши...
2808    <p>Сегодня в центре Киева, в ресторания ""Суши...
2809    <p>Сегодня в центре Киева, в ресторания ""Суши...
2811    <p>Сегодня в центре Киева, в ресторания ""Суши...
2813    <p>Сегодня в центре Киева, в ресторания ""Суши...
2814    <p>Сегодня в центре Киева, в ресторания ""Суши...
2815    <p>Сегодня в центре Киева, в ресторания ""Суши...
2817    <p>Сегодня в центре Киева, в ресторания ""Суши...
2819    <p>Сегодня в центре Киева, в ресторания ""Суши...
2820    <p>Сегодня в центре Киева, в ресторания ""Суши...
2824    <p>Сегодня в центре Киева, в ресторания ""Суши...
2826    <p>Сегодня в центре Киева, в ресторания ""Суши...
2842    <p>Сегодня в центре Киева, в ресторания ""Суши...
2843    <p>Сегодня в центре Киева, в ресторания ""Суши...
2844    <p>Сег

Кількість текстів, що починаються з тегу &lt;p&gt;, тобто відкриття абзацу

In [9]:
df['text'].str.startswith('<p>').sum()
updated_df = df.drop_duplicates(['text'])
updated_df['keyword_tone'].value_counts()
updated_df['keyword_role'].value_counts()

object       7020
subject       425
secondary      45
Name: keyword_role, dtype: int64

Забираємо HTML розмітку

In [10]:
updated_df['text'] = updated_df['text'].apply(lambda x: BeautifulSoup(str(x), "lxml").text.replace("\n", " "))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Забираємо VK розмітку

In [11]:
updated_df['text'] = updated_df['text'].apply(lambda x: re.sub(r'\[([^\|\]]+\|)?([^\]]+)\]',r'\2', x))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
updated_df['language'] = "unknown"
for index, row in updated_df.iterrows():
    try:
        updated_df['language'][index] = Detector(row["text"]).language.code    
    except:
        print(row["text"])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/ira/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


Виталий, ска, Виталик, дай пять.мы прекрасно сыграли эту партию)мне Сушия не нра, шо делать,спасай


Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


Вчера, приблизительно в 21.00 были в сушия в р-не Южного Вокзала (ул. Георгия Кирпы, 3) заказывали суши с копченным лососем deluxe- это не суши, а крашенная селедка...реально экономят, на вкус никак:( скандалить не хотели...но больше в сушия никогда не пойду, так и отравиться недолго...при стоимости 109 грн. лучше добавлю и пойду в Мураками.Что и Вам советую!!!Всем удачи


Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


...

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


Вечер суши #sushi #evening #japanese #food #rolls #philadelphia #instafood #delivery #yummy #sushitime #loveit #greatmood #greatday #home #sushiya #sumy #ukraine #суши #роллы #филадельфия #сушия


Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


Какао с маршмеллоу#какао#с#маршмеллоу #вкусно#нямням#хочу#mydream😘 #люблю#лайкі#взаимныелайки #Instagram#sushiya#лав#моялюбовьнавсегда


Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


Розподіл мов

In [13]:
updated_df["language"].value_counts()

ru         5036
uk         1078
en          440
bg          438
unknown     354
sr           38
da           18
ro           10
be            7
sco           7
uz            6
tr            4
mk            4
ky            4
cs            4
kl            3
sm            3
ie            3
tg            3
sw            2
nn            2
af            2
rm            2
sk            2
jw            2
tk            2
kk            2
pt            1
fo            1
zu            1
ln            1
na            1
lt            1
mi            1
ht            1
mn            1
es            1
pl            1
aa            1
zzp           1
no            1
Name: language, dtype: int64

Змінюємо negative на -1, а positive на 1, neutral на 0

In [14]:
mapper = {"neutral": 0, "negative": -1, "positive":1} 
updated_df["keyword_tone"] = updated_df["keyword_tone"].apply(lambda x: mapper[x])
updated_df.head()


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,title,text,keyword_role,keyword_tone,source_type,source_url,language
0,Какой у вас в Харькове Сушия? У нас в Одессе к...,Какой у вас в Харькове Сушия? У нас в Одессе к...,object,-1,socialnetworksource,http://instagram.com/,ru
1,"Меня качество еды в Сушия устраивает, но не ве...","Меня качество еды в Сушия устраивает, но не ве...",object,-1,socialnetworksource,http://youtube.com/,ru
2,"[club30459962|Сушия – сучасна японська кухня],...","Сушия – сучасна японська кухня, тогда, получае...",object,-1,socialnetworksource,http://vk.com/,ru
3,​Ни при каких не заказывайте доставку,Отзыв о Сушия: Ни при каких не заказывайте до...,object,-1,socialnetworksource,http://www.otzyvua.net/,ru
4,"Сушия, на час не на 2.","Сушия, на час не на 2.",object,-1,socialnetworksource,http://vk.com/,sr


In [15]:
df_ru = updated_df[updated_df["language"] == "ru"]
df_ru = df_ru[(df_ru["keyword_tone"] == -1) | (df_ru["keyword_tone"] == 1)]

In [16]:
data_train, data_test, labels_train, labels_test = train_test_split(df_ru["text"], df_ru["keyword_tone"], test_size=0.1, random_state=42, stratify=df_ru["keyword_tone"])
mdl = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1, 3), max_features=10000)), ('lr', LogisticRegression(C=1, n_jobs=-1))])
mdl.fit(data_train, labels_train)
mdl.score(data_test, labels_test)

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


0.88736263736263732

In [17]:
updated_df[updated_df["language"] == "da"].iloc[8]

title           #vsco #vscocam #vscolove #vscolive #vscolikes ...
text            #vsco #vscocam #vscolove #vscolive #vscolikes ...
keyword_role                                               object
keyword_tone                                                    1
source_type                                   socialnetworksource
source_url                                  http://instagram.com/
language                                                       da
Name: 6636, dtype: object

In [46]:

splited = updated_df.copy()
splited["text"]=splited["text"].apply( lambda x : sent_tokenize(x))


In [47]:
splited.head()

,title,text,keyword_role,keyword_tone,source_type,source_url,language
0,Какой у вас в Харькове Сушия? У нас в Одессе к...,"[Какой у вас в Харькове Сушия?, У нас в Одессе...",object,-1,socialnetworksource,http://instagram.com/,ru
1,"Меня качество еды в Сушия устраивает, но не ве...","[Меня качество еды в Сушия устраивает, но не в...",object,-1,socialnetworksource,http://youtube.com/,ru
2,"[club30459962|Сушия – сучасна японська кухня],...","[Сушия – сучасна японська кухня, тогда, получа...",object,-1,socialnetworksource,http://vk.com/,ru
3,​Ни при каких не заказывайте доставку,[ Отзыв о Сушия: Ни при каких не заказывайте д...,object,-1,socialnetworksource,http://www.otzyvua.net/,ru
4,"Сушия, на час не на 2.","[Сушия, на час не на 2.]",object,-1,socialnetworksource,http://vk.com/,sr


In [48]:
#splited["keyword"] = "Сушия"


In [49]:
# Запамятовувати забарвлення попередніх

,title,text,keyword_role,keyword_tone,source_type,source_url,language,keyword
0,Какой у вас в Харькове Сушия? У нас в Одессе к...,"[Какой у вас в Харькове Сушия?, У нас в Одессе...",object,-1,socialnetworksource,http://instagram.com/,ru,Сушия
1,"Меня качество еды в Сушия устраивает, но не ве...","[Меня качество еды в Сушия устраивает, но не в...",object,-1,socialnetworksource,http://youtube.com/,ru,Сушия
2,"[club30459962|Сушия – сучасна японська кухня],...","[Сушия – сучасна японська кухня, тогда, получа...",object,-1,socialnetworksource,http://vk.com/,ru,Сушия
3,​Ни при каких не заказывайте доставку,[ Отзыв о Сушия: Ни при каких не заказывайте д...,object,-1,socialnetworksource,http://www.otzyvua.net/,ru,Сушия
4,"Сушия, на час не на 2.","[Сушия, на час не на 2.]",object,-1,socialnetworksource,http://vk.com/,sr,Сушия


In [50]:
splited["text"]=splited["text"].apply( lambda x : [i for i in x if i.find("Сушия") != -1])
splited.head()


,title,text,keyword_role,keyword_tone,source_type,source_url,language,keyword
0,Какой у вас в Харькове Сушия? У нас в Одессе к...,[Какой у вас в Харькове Сушия?],object,-1,socialnetworksource,http://instagram.com/,ru,Сушия
1,"Меня качество еды в Сушия устраивает, но не ве...","[Меня качество еды в Сушия устраивает, но не в...",object,-1,socialnetworksource,http://youtube.com/,ru,Сушия
2,"[club30459962|Сушия – сучасна японська кухня],...","[Сушия – сучасна японська кухня, тогда, получа...",object,-1,socialnetworksource,http://vk.com/,ru,Сушия
3,​Ни при каких не заказывайте доставку,[ Отзыв о Сушия: Ни при каких не заказывайте д...,object,-1,socialnetworksource,http://www.otzyvua.net/,ru,Сушия
4,"Сушия, на час не на 2.","[Сушия, на час не на 2.]",object,-1,socialnetworksource,http://vk.com/,sr,Сушия


In [51]:
splited["text"]=splited["text"].apply( lambda x : '.'.join(x))
df_ru1 = splited[splited["language"] == "ru"]
df_ru1 = df_ru1[(df_ru1["keyword_tone"] == -1) | (df_ru1["keyword_tone"] == 1)]


data_train, data_test, labels_train, labels_test = train_test_split(df_ru1["text"], df_ru1["keyword_tone"], test_size=0.1, random_state=42, stratify=df_ru1["keyword_tone"])
mdl = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1, 3), max_features=10000)), ('lr', LogisticRegression(C=1, n_jobs=-1))])
mdl.fit(data_train, labels_train)
mdl.score(data_test, labels_test)

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


0.68956043956043955

In [53]:
splited["text"][0]

'Какой у вас в Харькове Сушия?'